# The add_interface method <3

Using data from the Buzsaki data share: https://buzsakilab.nyumc.org/datasets/WatsonBO/BWRat17/BWRat17_121712/

In [ ]:
from pathlib import Path
from pprint import pprint

from nwb_conversion_tools import NWBConverter, spec
from nwb_conversion_tools.interfaces import list_interfaces
import pynwb


The base path is the root of a single data directory -- later we will apply it to any number of directories :)

In [ ]:
base_path = Path('~/Desktop/watson/BWRat17/BWRat17_121712').expanduser()

In [ ]:
converter = NWBConverter(base_path)

# Specify Metadata
We specify the general structure and location of the data and metadata within a folder, rather than loading it explicitly


Let's tell it where to find that medaaaadadaaaa. Our session id is the full name of a file named, in this case `'BWRat17_121712.lfp'`, and we want to store this as some metadata nested within `'NWBFile'` -- so let's tell the converter that!

We use the `spec.Path` specifier to extract metadata from a path

In [ ]:
sid_spec = spec.Path('{NWBFile[identifier]}.lfp')
sid_spec.parse(base_path)

Fabulous! now we can add that to our converter!!!

In [ ]:
converter.add_metadata(sid_spec)

Keep doin that huh? add some more metadata! This dataset uses the date as the `session_start_time` parameter. As a bonus, we can specify multiple pieces of metadata in a single spec -- they get merged at the end don't worry!

In [ ]:
start_spec = spec.Path('{Subject[subject_id]}_{NWBFile[session_start_time]}.lfp')
pprint(start_spec.parse(base_path))
converter.add_metadata(start_spec)

Static metadata is also fine

In [ ]:
converter.add_metadata({'NWBFile':{'institution':'NYU'}})

# Specifying Interfaces


To see the available interfaces, we can just ask!

In [ ]:
# list all interfaces
pprint(list_interfaces()[0:10])

In [ ]:
# list interfaces of a specific category
pprint(list_interfaces('imaging'))

In [ ]:
# and get a specific interface
tiff_interface = list_interfaces('imaging', 'tiff')
tiff_interface

In [ ]:
# the search depends on the class attributes
print('\n'.join((
    tiff_interface.interface_type, 
    tiff_interface.device_name))
     )

## Adding a Neuroscope Sorting Interface

Using that syntax, we can programmatically add an interface to our dataset spec. First we can query what parameters we need to add it

In [ ]:
converter.add_interface('sorting', 'neuroscope')

We can specify `folder_path`, `gain`, and some others, but `folder_path` is required..

Let's use another spec -- this time the `Glob` spec, which is sort of the opposite of the `Path` spec -- using a pattern and/or some metadata, specify a file. in this case the data is originally in the base directory, but that's no fun! let's put them in a subdirectory that uses the subject id just to make it a lil more fun.

In [ ]:
sub_path = base_path / "BWRat17_121712"
sub_path.mkdir(exist_ok=True)

matches = list(base_path.glob('*.clu*'))
matches.extend(list(base_path.glob('*.res*')))
matches.extend(list(base_path.glob('*.xml')))
matches.extend(list(base_path.glob('*.spk*')))

for match in matches:
    match.rename(sub_path / match.name)

Now we use the `Glob`!

In [ ]:
converter.add_interface(
    'sorting', 'neuroscope', 
    spec.Glob('folder_path', 
              '{NWBFile[identifier]}'))

In [ ]:
output_path = Path('~').expanduser() / 'test_nwb.nwb'
converter.run_conversion(nwbfile_path=str(output_path))

Did it work? 

In [ ]:
io = pynwb.NWBHDF5IO(str(output_path), 'r')
nwbfile_in = io.read()
print(nwbfile_in)

Uh oh! looks like the nwb library quietly errored in its attempt to convert our timestamp string `'121712'`! we'll need to add postprocessing/reformatting to `spec` objects (should be np just another argument yno) Other than that we got it tho. It also looks like pynwb overwrites `identifier` with a hash, which I think is as-advertised, but this too passes silently! 

how about our spikes?

In [ ]:

print(nwbfile_in.units)

Hooray!

# Saving/Restoring Conversion Config

Assuming we have a quasi-stable structure to at least some of our data, we might want to re-use this configuration. We can save and load our converter configurations!

In [ ]:
json_path = Path()/'converter_config.json'

converter_json = converter.to_json(json_path)

pprint(converter_json)

Make a new one using the `from_json` method!

In [ ]:
converter_2 = NWBConverter.from_json(json_path)

assert converter_json == converter_2.to_json()

# Convert many folders

Now that we've got a converter, goode and true, it's easy to apply it to many directories :)



In [ ]:
# 1 sec.//.
# converter_2.convert_many(base_path.parent.glob('*'))

# TODO

* Now that we've got an abstract representation of a dataset, we should be able to save it, load it, etc.
* It is also trivial to then apply it to a list of directories by making a simple `apply`-like method.  
* ??? lots more development ???

like this is what i'm on about:
```
converter = NWBConverter.from_spec('spec_file.pck')
converter.apply('/data/experiments/*')
```